# Javascripts and how it works for the weather https://weather-forecast-d6l3.onrender.com

## Component 1: DOM ELEMENTS (The controls)



```javascript
const elements = {
    btn: document.getElementById("getForecastBtn"),      // Get Forecast button
    select: document.getElementById("citySelect"),       // City dropdown
    section: document.getElementById("forecastSection"), // Forecast container
    grid: document.getElementById("weatherGrid"),        // Weather cards container
    cityName: document.getElementById("forecastCityName"), // City name display
    cityIcon: document.getElementById("forecastCityIcon"), // Flag emoji display
    overlay: document.getElementById("loadingOverlay")   // Loading spinner
};
```

### What it does
- Grabs HTML elements once when page loads
- Stores references so we don't search the DOM repeatedly
- Think of it as "Here are all my remomote controls".

The index.html has the btn, select, section, grid, etc then over to Javascript, to grab them, we use

**Element**: document.get**ElementById** ("getForecastBtn"),

### Visual Connection
``` javascript
HTML                              JavaScript
─────────────────────────────────────────────────
<select id="citySelect">    →    elements.select
<button id="getForecastBtn"> →   elements.btn
<div id="weatherGrid">      →    elements.grid
```

## 📦 Component 2: Data Libraries (The Knowledge)
### Weather Icon
```javascript
const ICONS_IOS = {
    clear: "☀️",        // API returns "clear" → show sun
    pcloudy: "⛅",      // API returns "pcloudy" → show partly cloudy
    rain: "🌧️",         // API returns "rain" → show rain
    snow: "❄️",         // API returns "snow" → show snowflake
    // ... more mappings
    default: "❓"       // Unknown code → show question mark
};
```
### Weather label

```javascript
const WEATHER_DETAILS = {
    clear: "Clear",              // API code → Human readable
    pcloudy: "Partly Cloudy",
    rain: "Rain",
    snow: "Snow",
    default: "Unknown"
};
```
### City Backround 
```javascript
const cityBG = {
    paris: "images/paris.jpg",   // City key → Image path
    london: "images/london.jpg",
    prague: "images/prague.jpg",
    // ... more cities
};
```
## How do they work together?
```text
API returns: "pcloudy"
     │
     ├─→ ICONS_IOS["pcloudy"]      → "⛅"
     │
     └─→ WEATHER_DETAILS["pcloudy"] → "Partly Cloudy"

HTML data-bg="paris"
     │
     └─→ cityBG["paris"]           → "images/paris.jpg"
```

## Component 3: Configuration
```javascript
const CONFIG = {
    TRANSITION: 800,      // Background fade duration (milliseconds)
    API_PRODUCT: "civil"  // 7timer API product type
};
```

### Why This Matters:

    - Single source of truth for timing/settings
   -  Change once, applies everywhere
    - Easy to tweak without hunting through code

## 📦 Component 4: Background Changer
```javascript
let bgLayer = null;  // Track current background layer

function changeBackground(imagePath) {
    // Step 1: Preload image
    const img = new Image();
    img.src = imagePath;
    
    img.onload = () => {
        // Step 2: Create new layer
        const newLayer = document.createElement("div");
        newLayer.style.cssText = `
            position: fixed;
            inset: 0;
            background: url('${imagePath}') center/cover no-repeat;
            z-index: -100;
            opacity: 0;
            transition: opacity 0.8s ease;
        `;
        document.body.appendChild(newLayer);
        
        // Step 3: Fade in
        requestAnimationFrame(() => {
            requestAnimationFrame(() => {
                newLayer.style.opacity = "1";
            });
        });
        
        // Step 4: Cleanup old layer
        setTimeout(() => {
            if (bgLayer) bgLayer.remove();
            bgLayer = newLayer;
        }, 850);
    };
}
```

### Visual timeline
```text
Time:     0ms        100ms       400ms       800ms       850ms
          │           │           │           │           │
Image:    [preloading]●loaded     
Layer:                [created]──[fading in]──[fully visible]
Old Layer:            [visible]──[visible]────[visible]───●removed
```
### Why requestAnimationFrame Twice?
requestAnimationFrame(() => {        // Wait for browser to add element
    requestAnimationFrame(() => {    // Wait for browser to register initial state
        newLayer.style.opacity = "1"; // NOW trigger transition
    });
});

- First rAF: Browser adds element to DOM
- Second rAF: Browser calculates initial styles (opacity: 0)
- Then: We change to opacity: 1 → transition animates!

## JavaScript Architecture Explained 🏗️

Let me break down every component and how they work together like a machine.
🎯 Big Picture Overview

text

┌─────────────────────────────────────────────────────────────────┐
│                    USER INTERACTIONS                            │
├─────────────────────────────────────────────────────────────────┤
│  Select City          Click "Get Forecast"      Click "Change" │
│       │                       │                       │        │
│       ▼                       ▼                       ▼        │
│  updateCity()            handleGet()           hide section    │
│       │                       │                                │
│       ▼                       ├──────────┬────────────┐        │
│  changeBackground()           ▼          ▼            ▼        │
│                          show overlay  initMap()  loadWeather()│
│                               │          │            │        │
│                               ▼          ▼            ▼        │
│                          hide overlay  Leaflet    7timer API   │
│                                          │            │        │
│                                          ▼            ▼        │
│                                      Map Tiles   Weather Cards │
└─────────────────────────────────────────────────────────────────┘

📦 Component 1: DOM Elements (The Controls)

javascript

const elements = {
    btn: document.getElementById("getForecastBtn"),      // Get Forecast button
    select: document.getElementById("citySelect"),       // City dropdown
    section: document.getElementById("forecastSection"), // Forecast container
    grid: document.getElementById("weatherGrid"),        // Weather cards container
    cityName: document.getElementById("forecastCityName"), // City name display
    cityIcon: document.getElementById("forecastCityIcon"), // Flag emoji display
    overlay: document.getElementById("loadingOverlay")   // Loading spinner
};

What This Does:

    Grabs HTML elements once when page loads
    Stores references so we don't search the DOM repeatedly
    Think of it as: "Here are all my remote controls"

Visual Connection:

text

HTML                              JavaScript
─────────────────────────────────────────────────
<select id="citySelect">    →    elements.select
<button id="getForecastBtn"> →   elements.btn
<div id="weatherGrid">      →    elements.grid

📦 Component 2: Data Libraries (The Knowledge)
Weather Icons

javascript

const ICONS_IOS = {
    clear: "☀️",        // API returns "clear" → show sun
    pcloudy: "⛅",      // API returns "pcloudy" → show partly cloudy
    rain: "🌧️",         // API returns "rain" → show rain
    snow: "❄️",         // API returns "snow" → show snowflake
    // ... more mappings
    default: "❓"       // Unknown code → show question mark
};

Weather Labels

javascript

const WEATHER_DETAILS = {
    clear: "Clear",              // API code → Human readable
    pcloudy: "Partly Cloudy",
    rain: "Rain",
    snow: "Snow",
    default: "Unknown"
};

City Backgrounds

javascript

const cityBG = {
    paris: "images/paris.jpg",   // City key → Image path
    london: "images/london.jpg",
    prague: "images/prague.jpg",
    // ... more cities
};

### How They Work Together:

text

API returns: "pcloudy"
     │
     ├─→ ICONS_IOS["pcloudy"]      → "⛅"
     │
     └─→ WEATHER_DETAILS["pcloudy"] → "Partly Cloudy"

HTML data-bg="paris"
     │
     └─→ cityBG["paris"]           → "images/paris.jpg"

## 📦 Component 3: Configuration

```javascript

const CONFIG = {
    TRANSITION: 800,      // Background fade duration (milliseconds)
    API_PRODUCT: "civil"  // 7timer API product type
};
```

Why This Matters:

    Single source of truth for timing/settings
    Change once, applies everywhere
    Easy to tweak without hunting through code

## 📦 Component 4: Background Changer

```javascript

let bgLayer = null;  // Track current background layer

function changeBackground(imagePath) {
    // Step 1: Preload image
    const img = new Image();
    img.src = imagePath;
    
    img.onload = () => {
        // Step 2: Create new layer
        const newLayer = document.createElement("div");
        newLayer.style.cssText = `
            position: fixed;
            inset: 0;
            background: url('${imagePath}') center/cover no-repeat;
            z-index: -100;
            opacity: 0;
            transition: opacity 0.8s ease;
        `;
        document.body.appendChild(newLayer);
        
        // Step 3: Fade in
        requestAnimationFrame(() => {
            requestAnimationFrame(() => {
                newLayer.style.opacity = "1";
            });
        });
        
        // Step 4: Cleanup old layer
        setTimeout(() => {
            if (bgLayer) bgLayer.remove();
            bgLayer = newLayer;
        }, 850);
    };
}
```

Visual Timeline:

```text

Time:     0ms        100ms       400ms       800ms       850ms
          │           │           │           │           │
Image:    [preloading]●loaded     
Layer:                [created]──[fading in]──[fully visible]
Old Layer:            [visible]──[visible]────[visible]───●removed
```

Why requestAnimationFrame Twice?

```javascript

requestAnimationFrame(() => {        // Wait for browser to add element
    requestAnimationFrame(() => {    // Wait for browser to register initial state
        newLayer.style.opacity = "1"; // NOW trigger transition
    });
});
```
    First rAF: Browser adds element to DOM
    Second rAF: Browser calculates initial styles (opacity: 0)
    Then: We change to opacity: 1 → transition animates!

📦 Component 5: City Updater

```javascript
function updateCity() {
    // Get selected option
    const opt = elements.select.options[elements.select.selectedIndex];
    if (!opt || !opt.value) return;  // Exit if nothing selected
    
    // Update display
    elements.cityName.textContent = opt.dataset.name;  // "Paris"
    elements.cityIcon.textContent = opt.dataset.flag;  // "🇫🇷"
    
    // Change background
    const bgPath = cityBG[opt.dataset.bg];  // "images/paris.jpg"
    if (bgPath) changeBackground(bgPath);
}
### Data Flow:
```htmld
<option value="48.85,2.35" 
        data-bg="paris" 
        data-name="Paris" 
        data-flag="🇫🇷">
    Paris
</option>
```
### Example:
User selects "Paris"
        │
        ▼
opt.dataset.name → "Paris"      → elements.cityName.textContent
opt.dataset.flag → "🇫🇷"        → elements.cityIcon.textContent
opt.dataset.bg   → "paris"      → cityBG["paris"] → "images/paris.jpg"
                                        │
                                        ▼
                                changeBackground("images/paris.jpg")

## Component 6: Leaflet Map
```javascript
let map = null;    // Map instance (reusable)
let marker = null; // Marker instance (reusable)

function initLeafletMap(lat, lon) {
    const mapContainer = document.getElementById("map");
    if (!mapContainer) return;
    
    if (!map) {
        // FIRST TIME: Create map
        map = L.map("map").setView([lat, lon], 7);
        
        // Add tile layer (the actual map images)
        L.tileLayer("https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png", {
            maxZoom: 18,
            attribution: "© OpenStreetMap"
        }).addTo(map);
        
        // Add marker
        marker = L.marker([lat, lon]).addTo(map);
    } else {
        // SUBSEQUENT TIMES: Just move the view
        map.setView([lat, lon], 7);
        marker.setLatLng([lat, lon]);
    }
    
    // Fix rendering after container becomes visible
    setTimeout(() => map.invalidateSize(), 400);
}

### Why map = null Check?
```text
First call:     map === null    → Create new map
Second call:    map !== null    → Reuse existing map, just move it
Third call:     map !== null    → Reuse existing map, just move it
```
### Why invalidateSize()?

``text
Problem:  Map container was hidden (display: none)
          Leaflet calculated size as 0x0 pixels
          
Solution: After container is visible, tell Leaflet:
          "Hey, recalculate your size!"
```

## 📦 Component 7: Weather Loader (The Brain)
```javascript
async function loadWeather(lat, lon) {
    // Step 1: Build API URL
    const url = `https://www.7timer.info/bin/api.pl?lon=${lon}&lat=${lat}&product=civil&output=json`;
    
    // Step 2: Fetch data
    let response, data;
    try {
        response = await fetch(url);
        const text = await response.text();
        data = JSON.parse(text);
    } catch (err) {
        elements.grid.innerHTML = `<div class="error-box">Error loading data</div>`;
        return;
    }
    
    // Step 3: Clear old cards
    elements.grid.innerHTML = "";
    
    // Step 4: Process each day
    data.dataseries.slice(0, 7).forEach((day, index) => {
        // ... create card for each day
    });
}
```

### Data flow:
```text
API URL
   │
   ▼
┌──────────────────────────────────────────────────────┐
│ https://www.7timer.info/bin/api.pl                   │
│   ?lon=2.35                                          │
│   &lat=48.85                                         │
│   &product=civil                                     │
│   &output=json                                       │
└──────────────────────────────────────────────────────┘
   │
   ▼
┌──────────────────────────────────────────────────────┐
│ {                                                    │
│   "product": "civil",                                │
│   "dataseries": [                                    │
│     { "temp2m": 9, "weather": "pcloudy", ... },      │
│     { "temp2m": 10, "weather": "mcloudy", ... },     │
│     ...                                              │
│   ]                                                  │
│ }                                                    │
└──────────────────────────────────────────────────────┘
   │
   ▼
.slice(0, 7) → Take first 7 days
   │
   ▼
.forEach() → Create a card for each day
```

### Card creation deep dive

```javascript
data.dataseries.slice(0, 7).forEach((day, index) => {
    // Get date
    const date = new Date();
    date.setDate(date.getDate() + index);  // Today + 0, +1, +2...
    
    // Get weather code
    const rawCode = day.weather;           // "pcloudy"
    const baseCode = rawCode.replace(/(day|night)$/i, ""); // "pcloudy"
    
    // Look up icon and label
    const icon = ICONS_IOS[baseCode];      // "⛅"
    const label = WEATHER_DETAILS[baseCode]; // "Partly Cloudy"
    
    // Create HTML card
    const card = document.createElement("div");
    card.className = "weather-card weather-animate";
    card.style.animationDelay = `${index * 0.1}s`;  // Stagger animation
    card.innerHTML = `
        <div class="w-day">${weekday}</div>
        <div class="w-icon">${icon}</div>
        <div class="w-temp">${temp}°C</div>
        ...
    `;
    
    // Add to grid
    elements.grid.appendChild(card);
});
```

### Animation stagger:
```text
Card 0: delay = 0.0s   ████████░░░░░░░░
Card 1: delay = 0.1s   ░░████████░░░░░░
Card 2: delay = 0.2s   ░░░░████████░░░░
Card 3: delay = 0.3s   ░░░░░░████████░░
...
```

## 📦 Component 8: Main Handler|

```javascript
async function handleGet() {
    // Validate
    const val = elements.select.value;
    if (!val) {
        alert("Please select a city first!");
        return;
    }
    
    // Parse coordinates
    const [lat, lon] = val.split(",").map(Number);
    // "48.85,2.35" → ["48.85", "2.35"] → [48.85, 2.35]
    
    // Show loading
    elements.overlay.classList.add("active");
    
    // Show forecast section
    elements.section.classList.remove("hidden");
    
    // Scroll to forecast
    setTimeout(() => {
        elements.section.scrollIntoView({ behavior: "smooth" });
    }, 100);
    
    // Load data (parallel operations)
    initLeafletMap(lat, lon);
    await loadWeather(lat, lon);
    
    // Hide loading
    elements.overlay.classList.remove("active");
}
```

```text
User clicks "Get Forecast"
        │
        ▼
    ┌───────────┐
    │ Validate  │ → If no city selected, alert & stop
    └─────┬─────┘
          │
          ▼
    ┌───────────────┐
    │ Show Loading  │ → Spinner appears
    └───────┬───────┘
            │
            ▼
    ┌───────────────┐
    │ Show Section  │ → Forecast area becomes visible
    └───────┬───────┘
            │
            ▼
    ┌───────────────┐
    │ Scroll Down   │ → Page smoothly scrolls
    └───────┬───────┘
            │
     ┌──────┴──────┐
     ▼             ▼
┌─────────┐   ┌───────────┐
│ Init Map│   │ Load API  │  ← These run together!
└────┬────┘   └─────┬─────┘
     │              │
     ▼              ▼
  Map shows    Cards appear
            │
            ▼
    ┌───────────────┐
    │ Hide Loading  │ → Spinner disappears
    └───────────────┘
    ```

    ## Component 9: Initialiser 
    ```javascript
    document.addEventListener("DOMContentLoaded", () => {
    console.log("🌤️ App initialized");
    
    // Verify elements exist
    Object.entries(elements).forEach(([key, el]) => {
        if (el) console.log(`✅ ${key}`);
        else console.error(`❌ ${key} not found`);
    });
    
    // Wire up events
    elements.select.addEventListener("change", updateCity);
    elements.btn.addEventListener("click", handleGet);
    
    document.getElementById("changeCityBtn").addEventListener("click", () => {
        elements.section.classList.add("hidden");
        window.scrollTo({ top: 0, behavior: "smooth" });
    });
});
```

## What DOMContentLoaded does:
```text
Browser loads HTML
        │
        ▼ (parsing...)
        │
<script> tag found
        │
        ▼
Script starts executing...
BUT! DOM might not be fully built yet!
        │
        ▼
DOMContentLoaded fires ← "All HTML elements are ready!"
        │
        ▼
NOW safe to grab elements and attach events
```

## Complete Flow: User Selects Paris and Gets Forecast
```text
1. User selects "Paris" from dropdown
   │
   ▼
2. "change" event fires → updateCity()
   │
   ├── Display "🇫🇷 Paris"
   │
   └── changeBackground("images/paris.jpg")
       │
       └── Paris background fades in
   
3. User clicks "Get Forecast"
   │
   ▼
4. "click" event fires → handleGet()
   │
   ├── Parse "48.85,2.35" → lat=48.85, lon=2.35
   │
   ├── Show loading spinner
   │
   ├── Show forecast section
   │
   ├── Scroll down
   │
   ├── initLeafletMap(48.85, 2.35)
   │   │
   │   └── Map shows Paris area
   │
   ├── await loadWeather(48.85, 2.35)
   │   │
   │   ├── Fetch from 7timer API
   │   │
   │   ├── Parse JSON response
   │   │
   │   └── Create 7 weather cards
   │       │
   │       └── Cards animate in (staggered)
   │
   └── Hide loading spinner

5. User sees: Paris background + 7-day forecast + map
``

## How everything connects
```text
┌─────────────────────────────────────────────────────────────────┐
│                         HTML                                    │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐        │
│  │ <select> │  │ <button> │  │ <section>│  │  <div    │        │
│  │ cityList │  │ getBtn   │  │ forecast │  │  #map>   │        │
│  └────┬─────┘  └────┬─────┘  └────┬─────┘  └────┬─────┘        │
└───────┼─────────────┼─────────────┼─────────────┼───────────────┘
        │             │             │             │
        ▼             ▼             ▼             ▼
┌───────────────────────────────────────────────────────────────┐
│                      elements {}                               │
│         (References to all HTML elements)                      │
└───────────────────────────────────────────────────────────────┘
        │             │             │             │
        │             │             │             │
┌───────┼─────────────┼─────────────┼─────────────┼───────────────┐
│       │   EVENT LISTENERS         │             │               │
│       ▼             ▼             │             │               │
│  ┌─────────┐   ┌─────────┐       │             │               │
│  │onChange │   │ onClick │       │             │               │
│  └────┬────┘   └────┬────┘       │             │               │
│       │             │             │             │               │
│       ▼             ▼             │             │               │
│ updateCity()   handleGet()        │             │               │
│       │             │             │             │               │
│       │             ├─────────────┼─────────────┤               │
│       │             ▼             ▼             ▼               │
│       │      loadWeather()   show/hide    initLeafletMap()     │
│       │             │         section           │               │
│       │             │             │             │               │
│       ▼             ▼             │             │               │
│ changeBackground()  │             │             │               │
│       │             │             │             │               │
│       │        ┌────┴────┐       │             │               │
│       │        ▼         ▼       │             ▼               │
│       │   ICONS_IOS  WEATHER_    │        Leaflet.js           │
│       │              DETAILS     │        (external)           │
│       │                          │             │               │
│       ▼                          ▼             ▼               │
│   body::before              weatherGrid     #map               │
│   (CSS variable)            (DOM cards)    (tiles)             │
└───────────────────────────────────────────────────────────────┘
                              │
                              ▼
                    ┌─────────────────┐
                    │    SCREEN       │
                    │                 │
                    │ ┌─────────────┐ │
                    │ │ Background  │ │
                    │ │ (city img)  │ │
                    │ │             │ │
                    │ │ ┌─────────┐ │ │
                    │ │ │ Cards   │ │ │
                    │ │ └─────────┘ │ │
                    │ │ ┌─────────┐ │ │
                    │ │ │   Map   │ │ │
                    │ │ └─────────┘ │ │
                    │ └─────────────┘ │
                    └─────────────────┘
                    ```

JavaScript Architecture Explained 🏗️

Let me break down every component and how they work together like a machine.
🎯 Big Picture Overview

text

┌─────────────────────────────────────────────────────────────────┐
│                    USER INTERACTIONS                            │
├─────────────────────────────────────────────────────────────────┤
│  Select City          Click "Get Forecast"      Click "Change" │
│       │                       │                       │        │
│       ▼                       ▼                       ▼        │
│  updateCity()            handleGet()           hide section    │
│       │                       │                                │
│       ▼                       ├──────────┬────────────┐        │
│  changeBackground()           ▼          ▼            ▼        │
│                          show overlay  initMap()  loadWeather()│
│                               │          │            │        │
│                               ▼          ▼            ▼        │
│                          hide overlay  Leaflet    7timer API   │
│                                          │            │        │
│                                          ▼            ▼        │
│                                      Map Tiles   Weather Cards │
└─────────────────────────────────────────────────────────────────┘

📦 Component 1: DOM Elements (The Controls)

javascript

const elements = {
    btn: document.getElementById("getForecastBtn"),      // Get Forecast button
    select: document.getElementById("citySelect"),       // City dropdown
    section: document.getElementById("forecastSection"), // Forecast container
    grid: document.getElementById("weatherGrid"),        // Weather cards container
    cityName: document.getElementById("forecastCityName"), // City name display
    cityIcon: document.getElementById("forecastCityIcon"), // Flag emoji display
    overlay: document.getElementById("loadingOverlay")   // Loading spinner
};

What This Does:

    Grabs HTML elements once when page loads
    Stores references so we don't search the DOM repeatedly
    Think of it as: "Here are all my remote controls"

Visual Connection:

text

HTML                              JavaScript
─────────────────────────────────────────────────
<select id="citySelect">    →    elements.select
<button id="getForecastBtn"> →   elements.btn
<div id="weatherGrid">      →    elements.grid

📦 Component 2: Data Libraries (The Knowledge)
Weather Icons

javascript

const ICONS_IOS = {
    clear: "☀️",        // API returns "clear" → show sun
    pcloudy: "⛅",      // API returns "pcloudy" → show partly cloudy
    rain: "🌧️",         // API returns "rain" → show rain
    snow: "❄️",         // API returns "snow" → show snowflake
    // ... more mappings
    default: "❓"       // Unknown code → show question mark
};

Weather Labels

javascript

const WEATHER_DETAILS = {
    clear: "Clear",              // API code → Human readable
    pcloudy: "Partly Cloudy",
    rain: "Rain",
    snow: "Snow",
    default: "Unknown"
};

City Backgrounds

javascript

const cityBG = {
    paris: "images/paris.jpg",   // City key → Image path
    london: "images/london.jpg",
    prague: "images/prague.jpg",
    // ... more cities
};

How They Work Together:

text

API returns: "pcloudy"
     │
     ├─→ ICONS_IOS["pcloudy"]      → "⛅"
     │
     └─→ WEATHER_DETAILS["pcloudy"] → "Partly Cloudy"

HTML data-bg="paris"
     │
     └─→ cityBG["paris"]           → "images/paris.jpg"

📦 Component 3: Configuration

javascript

const CONFIG = {
    TRANSITION: 800,      // Background fade duration (milliseconds)
    API_PRODUCT: "civil"  // 7timer API product type
};

Why This Matters:

    Single source of truth for timing/settings
    Change once, applies everywhere
    Easy to tweak without hunting through code

📦 Component 4: Background Changer

javascript

let bgLayer = null;  // Track current background layer

function changeBackground(imagePath) {
    // Step 1: Preload image
    const img = new Image();
    img.src = imagePath;
    
    img.onload = () => {
        // Step 2: Create new layer
        const newLayer = document.createElement("div");
        newLayer.style.cssText = `
            position: fixed;
            inset: 0;
            background: url('${imagePath}') center/cover no-repeat;
            z-index: -100;
            opacity: 0;
            transition: opacity 0.8s ease;
        `;
        document.body.appendChild(newLayer);
        
        // Step 3: Fade in
        requestAnimationFrame(() => {
            requestAnimationFrame(() => {
                newLayer.style.opacity = "1";
            });
        });
        
        // Step 4: Cleanup old layer
        setTimeout(() => {
            if (bgLayer) bgLayer.remove();
            bgLayer = newLayer;
        }, 850);
    };
}

Visual Timeline:

text

Time:     0ms        100ms       400ms       800ms       850ms
          │           │           │           │           │
Image:    [preloading]●loaded     
Layer:                [created]──[fading in]──[fully visible]
Old Layer:            [visible]──[visible]────[visible]───●removed

Why requestAnimationFrame Twice?

javascript

requestAnimationFrame(() => {        // Wait for browser to add element
    requestAnimationFrame(() => {    // Wait for browser to register initial state
        newLayer.style.opacity = "1"; // NOW trigger transition
    });
});

    First rAF: Browser adds element to DOM
    Second rAF: Browser calculates initial styles (opacity: 0)
    Then: We change to opacity: 1 → transition animates!

📦 Component 5: City Updater

javascript

function updateCity() {
    // Get selected option
    const opt = elements.select.options[elements.select.selectedIndex];
    if (!opt || !opt.value) return;  // Exit if nothing selected
    
    // Update display
    elements.cityName.textContent = opt.dataset.name;  // "Paris"
    elements.cityIcon.textContent = opt.dataset.flag;  // "🇫🇷"
    
    // Change background
    const bgPath = cityBG[opt.dataset.bg];  // "images/paris.jpg"
    if (bgPath) changeBackground(bgPath);
}

Data Flow:

html

<option value="48.85,2.35" 
        data-bg="paris" 
        data-name="Paris" 
        data-flag="🇫🇷">
    Paris
</option>

text

User selects "Paris"
        │
        ▼
opt.dataset.name → "Paris"      → elements.cityName.textContent
opt.dataset.flag → "🇫🇷"        → elements.cityIcon.textContent
opt.dataset.bg   → "paris"      → cityBG["paris"] → "images/paris.jpg"
                                        │
                                        ▼
                                changeBackground("images/paris.jpg")

📦 Component 6: Leaflet Map

javascript

let map = null;    // Map instance (reusable)
let marker = null; // Marker instance (reusable)

function initLeafletMap(lat, lon) {
    const mapContainer = document.getElementById("map");
    if (!mapContainer) return;
    
    if (!map) {
        // FIRST TIME: Create map
        map = L.map("map").setView([lat, lon], 7);
        
        // Add tile layer (the actual map images)
        L.tileLayer("https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png", {
            maxZoom: 18,
            attribution: "© OpenStreetMap"
        }).addTo(map);
        
        // Add marker
        marker = L.marker([lat, lon]).addTo(map);
    } else {
        // SUBSEQUENT TIMES: Just move the view
        map.setView([lat, lon], 7);
        marker.setLatLng([lat, lon]);
    }
    
    // Fix rendering after container becomes visible
    setTimeout(() => map.invalidateSize(), 400);
}

Why map = null Check?

text

First call:     map === null    → Create new map
Second call:    map !== null    → Reuse existing map, just move it
Third call:     map !== null    → Reuse existing map, just move it

Benefit: Creating a map is expensive. Reusing is fast!
Why invalidateSize()?

text

Problem:  Map container was hidden (display: none)
          Leaflet calculated size as 0x0 pixels
          
Solution: After container is visible, tell Leaflet:
          "Hey, recalculate your size!"

📦 Component 7: Weather Loader (The Brain)

javascript

async function loadWeather(lat, lon) {
    // Step 1: Build API URL
    const url = `https://www.7timer.info/bin/api.pl?lon=${lon}&lat=${lat}&product=civil&output=json`;
    
    // Step 2: Fetch data
    let response, data;
    try {
        response = await fetch(url);
        const text = await response.text();
        data = JSON.parse(text);
    } catch (err) {
        elements.grid.innerHTML = `<div class="error-box">Error loading data</div>`;
        return;
    }
    
    // Step 3: Clear old cards
    elements.grid.innerHTML = "";
    
    // Step 4: Process each day
    data.dataseries.slice(0, 7).forEach((day, index) => {
        // ... create card for each day
    });
}

Data Flow:

text

API URL
   │
   ▼
┌──────────────────────────────────────────────────────┐
│ https://www.7timer.info/bin/api.pl                   │
│   ?lon=2.35                                          │
│   &lat=48.85                                         │
│   &product=civil                                     │
│   &output=json                                       │
└──────────────────────────────────────────────────────┘
   │
   ▼
┌──────────────────────────────────────────────────────┐
│ {                                                    │
│   "product": "civil",                                │
│   "dataseries": [                                    │
│     { "temp2m": 9, "weather": "pcloudy", ... },      │
│     { "temp2m": 10, "weather": "mcloudy", ... },     │
│     ...                                              │
│   ]                                                  │
│ }                                                    │
└──────────────────────────────────────────────────────┘
   │
   ▼
.slice(0, 7) → Take first 7 days
   │
   ▼
.forEach() → Create a card for each day

Card Creation Deep Dive:

javascript

data.dataseries.slice(0, 7).forEach((day, index) => {
    // Get date
    const date = new Date();
    date.setDate(date.getDate() + index);  // Today + 0, +1, +2...
    
    // Get weather code
    const rawCode = day.weather;           // "pcloudy"
    const baseCode = rawCode.replace(/(day|night)$/i, ""); // "pcloudy"
    
    // Look up icon and label
    const icon = ICONS_IOS[baseCode];      // "⛅"
    const label = WEATHER_DETAILS[baseCode]; // "Partly Cloudy"
    
    // Create HTML card
    const card = document.createElement("div");
    card.className = "weather-card weather-animate";
    card.style.animationDelay = `${index * 0.1}s`;  // Stagger animation
    card.innerHTML = `
        <div class="w-day">${weekday}</div>
        <div class="w-icon">${icon}</div>
        <div class="w-temp">${temp}°C</div>
        ...
    `;
    
    // Add to grid
    elements.grid.appendChild(card);
});

Animation Stagger:

text

Card 0: delay = 0.0s   ████████░░░░░░░░
Card 1: delay = 0.1s   ░░████████░░░░░░
Card 2: delay = 0.2s   ░░░░████████░░░░
Card 3: delay = 0.3s   ░░░░░░████████░░
...

📦 Component 8: Main Handler

javascript

async function handleGet() {
    // Validate
    const val = elements.select.value;
    if (!val) {
        alert("Please select a city first!");
        return;
    }
    
    // Parse coordinates
    const [lat, lon] = val.split(",").map(Number);
    // "48.85,2.35" → ["48.85", "2.35"] → [48.85, 2.35]
    
    // Show loading
    elements.overlay.classList.add("active");
    
    // Show forecast section
    elements.section.classList.remove("hidden");
    
    // Scroll to forecast
    setTimeout(() => {
        elements.section.scrollIntoView({ behavior: "smooth" });
    }, 100);
    
    // Load data (parallel operations)
    initLeafletMap(lat, lon);
    await loadWeather(lat, lon);
    
    // Hide loading
    elements.overlay.classList.remove("active");
}

Timeline:

text

User clicks "Get Forecast"
        │
        ▼
    ┌───────────┐
    │ Validate  │ → If no city selected, alert & stop
    └─────┬─────┘
          │
          ▼
    ┌───────────────┐
    │ Show Loading  │ → Spinner appears
    └───────┬───────┘
            │
            ▼
    ┌───────────────┐
    │ Show Section  │ → Forecast area becomes visible
    └───────┬───────┘
            │
            ▼
    ┌───────────────┐
    │ Scroll Down   │ → Page smoothly scrolls
    └───────┬───────┘
            │
     ┌──────┴──────┐
     ▼             ▼
┌─────────┐   ┌───────────┐
│ Init Map│   │ Load API  │  ← These run together!
└────┬────┘   └─────┬─────┘
     │              │
     ▼              ▼
  Map shows    Cards appear
            │
            ▼
    ┌───────────────┐
    │ Hide Loading  │ → Spinner disappears
    └───────────────┘

📦 Component 9: Initializer

javascript

document.addEventListener("DOMContentLoaded", () => {
    console.log("🌤️ App initialized");
    
    // Verify elements exist
    Object.entries(elements).forEach(([key, el]) => {
        if (el) console.log(`✅ ${key}`);
        else console.error(`❌ ${key} not found`);
    });
    
    // Wire up events
    elements.select.addEventListener("change", updateCity);
    elements.btn.addEventListener("click", handleGet);
    
    document.getElementById("changeCityBtn").addEventListener("click", () => {
        elements.section.classList.add("hidden");
        window.scrollTo({ top: 0, behavior: "smooth" });
    });
});

What DOMContentLoaded Does:

text

Browser loads HTML
        │
        ▼ (parsing...)
        │
<script> tag found
        │
        ▼
Script starts executing...
BUT! DOM might not be fully built yet!
        │
        ▼
DOMContentLoaded fires ← "All HTML elements are ready!"
        │
        ▼
NOW safe to grab elements and attach events

Event Wiring:

text

elements.select  ──onChange──►  updateCity()
elements.btn     ──onClick───►  handleGet()
changeCityBtn    ──onClick───►  hide section + scroll up

🔄 Complete Flow: User Selects Paris and Gets Forecast

text

1. User selects "Paris" from dropdown
   │
   ▼
2. "change" event fires → updateCity()
   │
   ├── Display "🇫🇷 Paris"
   │
   └── changeBackground("images/paris.jpg")
       │
       └── Paris background fades in
   
3. User clicks "Get Forecast"
   │
   ▼
4. "click" event fires → handleGet()
   │
   ├── Parse "48.85,2.35" → lat=48.85, lon=2.35
   │
   ├── Show loading spinner
   │
   ├── Show forecast section
   │
   ├── Scroll down
   │
   ├── initLeafletMap(48.85, 2.35)
   │   │
   │   └── Map shows Paris area
   │
   ├── await loadWeather(48.85, 2.35)
   │   │
   │   ├── Fetch from 7timer API
   │   │
   │   ├── Parse JSON response
   │   │
   │   └── Create 7 weather cards
   │       │
   │       └── Cards animate in (staggered)
   │
   └── Hide loading spinner

5. User sees: Paris background + 7-day forecast + map

🧩 How Everything Connects

text

┌─────────────────────────────────────────────────────────────────┐
│                         HTML                                    │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐        │
│  │ <select> │  │ <button> │  │ <section>│  │  <div    │        │
│  │ cityList │  │ getBtn   │  │ forecast │  │  #map>   │        │
│  └────┬─────┘  └────┬─────┘  └────┬─────┘  └────┬─────┘        │
└───────┼─────────────┼─────────────┼─────────────┼───────────────┘
        │             │             │             │
        ▼             ▼             ▼             ▼
┌───────────────────────────────────────────────────────────────┐
│                      elements {}                               │
│         (References to all HTML elements)                      │
└───────────────────────────────────────────────────────────────┘
        │             │             │             │
        │             │             │             │
┌───────┼─────────────┼─────────────┼─────────────┼───────────────┐
│       │   EVENT LISTENERS         │             │               │
│       ▼             ▼             │             │               │
│  ┌─────────┐   ┌─────────┐       │             │               │
│  │onChange │   │ onClick │       │             │               │
│  └────┬────┘   └────┬────┘       │             │               │
│       │             │             │             │               │
│       ▼             ▼             │             │               │
│ updateCity()   handleGet()        │             │               │
│       │             │             │             │               │
│       │             ├─────────────┼─────────────┤               │
│       │             ▼             ▼             ▼               │
│       │      loadWeather()   show/hide    initLeafletMap()     │
│       │             │         section           │               │
│       │             │             │             │               │
│       ▼             ▼             │             │               │
│ changeBackground()  │             │             │               │
│       │             │             │             │               │
│       │        ┌────┴────┐       │             │               │
│       │        ▼         ▼       │             ▼               │
│       │   ICONS_IOS  WEATHER_    │        Leaflet.js           │
│       │              DETAILS     │        (external)           │
│       │                          │             │               │
│       ▼                          ▼             ▼               │
│   body::before              weatherGrid     #map               │
│   (CSS variable)            (DOM cards)    (tiles)             │
└───────────────────────────────────────────────────────────────┘
                              │
                              ▼
                    ┌─────────────────┐
                    │    SCREEN       │
                    │                 │
                    │ ┌─────────────┐ │
                    │ │ Background  │ │
                    │ │ (city img)  │ │
                    │ │             │ │
                    │ │ ┌─────────┐ │ │
                    │ │ │ Cards   │ │ │
                    │ │ └─────────┘ │ │
                    │ │ ┌─────────┐ │ │
                    │ │ │   Map   │ │ │
                    │ │ └─────────┘ │ │
                    │ └─────────────┘ │
                    └─────────────────┘

📝 Summary
|Component|	Purpose|	Analogy|
elements	HTML references	Remote controls
ICONS_IOS	Weather code → Emoji	Translation dictionary
WEATHER_DETAILS	Weather code → Label	Translation dictionary
cityBG	City → Image path	Photo album
changeBackground()	Swap backgrounds	Stage curtain
updateCity()	Update display on selection	Label printer
initLeafletMap()	Create/move map	GPS navigator
loadWeather()	Fetch & display weather	Weather reporter
handleGet()	Orchestrate everything	Orchestra conductor
DOMContentLoaded	Wire up events	Plugging in appliances